In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
train = pd.read_csv("titanic/train.csv")


In [3]:
class_survivor_table = pd.pivot_table(train, index=["Pclass"], values=["Survived"])
#class_survivor_table.plot(kind="barh")
class_survivor_table

,Survived
Pclass,
1,0.629630
2,0.472826
3,0.242363


In [4]:
sex_survivor_table = pd.pivot_table(train, index=["Sex"], values=["Survived"])
#sex_survivor_table.plot(kind="barh")
sex_survivor_table

,Survived
Sex,
female,0.742038
male,0.188908


Group Ages into ranges:

Rationale: Age and survival are not linearly related. Specifically, children under 10 had an outsized chance of survival (they take up less room on a raft).

In [6]:
def create_age_range(age):
    if str(age) == "nan":
        return "unknown"
    elif 0 <= age < 5: 
        return "05_under" 
    elif 5 <= age < 12: 
        return "12_under" 
    elif 12 <= age < 18: 
        return "12to18" 
    elif 18 <= age < 35: 
        return "18to35" 
    elif 35 <= age < 50: 
        return "35to50" 
    elif 50 <= age: 
        return "50_plus" 

    
train["AgeRange"] = train["Age"].map(create_age_range)


sex_agerange_survivor_table = pd.pivot_table(train, index=["AgeRange"], values=["Survived"])
#sex_agerange_survivor_table.plot(kind="barh")
sex_agerange_survivor_table

,Survived
AgeRange,
05_under,0.675000
12_under,0.428571
12to18,0.488889
18to35,0.368852
35to50,0.416149
50_plus,0.364865
unknown,0.293785


In [9]:
SibSp_survivor_table = pd.pivot_table(train, index=["SibSp"], values=["Survived"])
#SibSp_survivor_table.plot(kind="barh")
SibSp_survivor_table

,Survived
SibSp,
0,0.345395
1,0.535885
2,0.464286
3,0.250000
4,0.166667
5,0.000000
8,0.000000


In [10]:
Parch_survivor_table = pd.pivot_table(train, index=["Parch"], values=["Survived"])
#Parch_survivor_table.plot(kind="barh")
Parch_survivor_table

,Survived
Parch,
0,0.343658
1,0.550847
2,0.500000
3,0.600000
4,0.000000
5,0.200000
6,0.000000


In [30]:
def create_fare_range(fare):
    if 0 <= fare < 10: 
        return ".under10" 
    elif 10 <= fare < 20: 
        return "10to20" 
    elif 20 <= fare < 30: 
        return "20to30" 
    elif 30 <= fare: 
        return "30+" 
    else:    
        return "_?unknown"  
train["FareRange"] = train["Age"].map(create_fare_range)
Fare_survivor_table = pd.pivot_table(train, index=["FareRange"], values=["Survived"])
#Fare_survivor_table.plot(kind="barh")
Fare_survivor_table

,Survived
FareRange,
.under10,0.612903
10to20,0.401961
20to30,0.350000
30+,0.406061
_?unknown,0.293785


In [26]:
all_onboard = 2224
train_onboard = len(train.Survived)
test_onboard = all_onboard - train_onboard
test_scaleup = test_onboard / train_onboard

all_survived = 722
train_survived = sum(train.Survived)
test_survived = all_survived - train_survived
survived_scaleup = test_survived / train_survived

print("aboard = {} \ntrain_aboard = {} \ntest_aboard = {} \nonboard_scaleup = {}".format(all_onboard, train_onboard, test_onboard, test_scaleup))
print("")
print("survived = {} \ntrain_survived = {} \ntest_survived = {} \nsurvived_scaleup = {}".format(all_survived, train_survived, test_survived, survived_scaleup))

aboard = 2224 
train_aboard = 891 
test_aboard = 1333 
onboard_scaleup = 1.4960718294051627

survived = 722 
train_survived = 342 
test_survived = 380 
survived_scaleup = 1.1111111111111112


In [27]:
def classlive(Pclass):
    if Pclass == 1:
        return 0.62963
    elif Pclass == 2:
        return 0.472826
    elif Pclass == 3:
        return 0.242363

def sexlive(Sex):
    if Sex == "female":
        return 0.742038
    elif Sex == "male":
        return 0.188908
    
def agerangelive(AgeRange):
    if AgeRange == "05_under":
        return 0.675
    elif AgeRange == "12_under":
        return 0.428571
    elif AgeRange == "12to18":
        return 0.488889
    elif AgeRange == "18to35":
        return 0.368852
    elif AgeRange == "35to50":
        return 0.416149
    elif AgeRange == "50_plus":
        return 0.364865
    else:
        return 0.293785

def farerangelive(farerange):
    if farerange == "10to20":
        return 0.401961
    elif farerange == "20to30":
        return 0.350000
    elif farerange == "30+":
        return 0.406061
    elif farerange == "_?unknown":
        return 0.293785
    elif farerange == "_under10":
        return 0.612903

train["farerangelives"] = train.FareRange.map(farerangelive)
train["classlives"] = train.Pclass.map(classlive)
train["sexlives"] = train.Sex.map(sexlive)
train["agerangelives"] = train.AgeRange.map(agerangelive)
train["Score"] = train.farerangelives * train.classlives * train.sexlives * train.agerangelives
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeRange,FareRange,TicketPrice,farerangelives,classlives,sexlives,agerangelives,Score
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,18to35,20to30,< $10,0.350000,0.242363,0.188908,0.368852,0.005911
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,35to50,30+,> $30,0.406061,0.629630,0.742038,0.416149,0.078950
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,18to35,20to30,< $10,0.350000,0.242363,0.742038,0.368852,0.023217
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,35to50,30+,> $30,0.406061,0.629630,0.742038,0.416149,0.078950
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,35to50,30+,< $10,0.406061,0.242363,0.188908,0.416149,0.007737


In [31]:
test = pd.read_csv("titanic/test.csv")
test["FareRange"] = test.Age.map(create_fare_range)
test["AgeRange"] = test["Age"].map(create_age_range)
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FareRange,AgeRange
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,30+,18to35
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,30+,35to50
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,30+,50_plus
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,20to30,18to35
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,20to30,18to35


In [33]:
test["farerangelives"] = test.FareRange.map(farerangelive)
test["classlives"] = test.Pclass.map(classlive)
test["sexlives"] = test.Sex.map(sexlive)
test["agerangelives"] = test.AgeRange.map(agerangelive)
test["Score"] = test.farerangelives * test.classlives * test.sexlives * test.agerangelives
test = test.sort(columns=["Score"], ascending=False)

In [34]:
count = len(test)
n = range(len(test))
limit = int(len(test) * 0.33333)
test.index = range(len(test))
test["rank"] = test.index

In [35]:
def survivors(rank):
    if rank < limit:
        return 1
    else:
        return 0

test["Survived"] = test["rank"].map(survivors)


In [ ]:
test1 = test[['PassengerId','Survived']]
test1.to_csv("titanic/ID_survival.csv", index=False)